In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [2]:
# DATA_SET = './datasets/aclimdb.csv'
DATA_SET = './IMDBDataset.csv'
df_gru =pd.read_csv(DATA_SET)
df_gru

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [3]:
train_set = 45000
max_len_text = 2000

df_gru['sentiment'] = df_gru['sentiment'].replace('positive', 1)
df_gru['sentiment'] = df_gru['sentiment'].replace('negative', 0)

In [4]:
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(df_gru['review'])
vector = tokenizer.texts_to_sequences(df_gru['review'])

In [5]:
train_data = np.array(vector)[:train_set]
train_label = (np.array(df_gru['sentiment'])[:train_set])
test_data = np.array(vector)[train_set:]
test_label = (np.array(df_gru['sentiment'])[train_set:])

C:\Users\Raunak\AppData\Local\Temp\ipykernel_13992\2786445468.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  train_data = np.array(vector)[:train_set]
C:\Users\Raunak\AppData\Local\Temp\ipykernel_13992\2786445468.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  test_data = np.array(vector)[train_set:]


In [6]:
padded_train = pad_sequences(train_data, maxlen=max_len_text)
padded_test = pad_sequences(test_data, maxlen=max_len_text)

In [7]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(13000, 16, input_length=max_len_text))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, dropout=0.2)))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 2000, 16)          208000    
_________________________________________________________________
bidirectional (Bidirectional (None, 128)               41472     
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 249,601
Trainable params: 249,601
Non-trainable params: 0
_________________________________________________________________


In [8]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [11]:
padded_train


array([[   0,    0,    0, ...,  125, 4103,  486],
       [   0,    0,    0, ..., 1977,   69,  221],
       [   0,    0,    0, ...,   63,   16,  350],
       ...,
       [   0,    0,    0, ..., 1802, 1355,   22],
       [   0,    0,    0, ...,    7,  156,  156],
       [   0,    0,    0, ...,    4,    1,  181]])

In [12]:
history = model.fit(padded_train, train_label, epochs=3, validation_data=(padded_test, test_label))

Epoch 1/3
1407/1407 [==============================] - 6331s 4s/step - loss: 0.5211 - accuracy: 0.7285 - val_loss: 0.3723 - val_accuracy: 0.8450
Epoch 2/3
1407/1407 [==============================] - 5858s 4s/step - loss: 0.2524 - accuracy: 0.9034 - val_loss: 0.2978 - val_accuracy: 0.8920
Epoch 3/3
1407/1407 [==============================] - 5003s 4s/step - loss: 0.2117 - accuracy: 0.9199 - val_loss: 0.2704 - val_accuracy: 0.8846


In [ ]:
from sklearn.metrics import classification_report

predictions = model.predict(padded_test)
predictions = predictions.argmax(axis=None)
print(classification_report(test_label.argmax(axis= None), predictions))